<a href="https://colab.research.google.com/github/sachinkun21/PyTorch/blob/master/Deploying_PyTorch_Model_in_Python_via_a_REST_API_with_FLASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, I will deploy a PyTorch model using Flask and expose a REST API for model inference. In particular, we will deploy a pretrained DenseNet 121 model which detects the image.

This represents the first in a series of tutorials on deploying PyTorch models in production. Using Flask in this way is by far the easiest way to start serving your PyTorch models, but it will not work for a use case with high performance requirements. For that:

- If you’re already familiar with TorchScript, you can jump straight into Loading a TorchScript Model in C++ tutorial.
- If you first need a refresher on TorchScript, check out Intro a TorchScript tutorial.

### **Step1: API Definition**
We will first define our API endpoints, the request and response types. Our API endpoint will be at /predict which takes HTTP POST requests with a file parameter which contains the image. The response will be of JSON response containing the prediction:

In [2]:
{"class_id":  "n02124075", "class_name": "Egyptian_cat"}

{'class_id': 'n02124075', 'class_name': 'Egyptian_cat'}

#### **Step2: Install and import Dependencies**

In [0]:
# $ pip install Flask==1.0.3 torchvision-0.3.0
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
  return 'Hello World!'
  

Save the above snippet in a file called app.py and you can now run a Flask development server by typing:

`$ FLASK_ENV=development FLASK_APP=app.py flask run`

When you visit http://localhost:5000/ in your web browser, you will be greeted with Hello World! text



### **Step 3: Modifying the Endpoint:**
We will make slight changes to the above snippet, so that it suits our API definition. First, we will rename the method to predict. We will update the endpoint path to /predict. Since the image files will be sent via HTTP POST requests, we will update it so that it also accepts only POST requests.


We will also change the response type, so that it returns a JSON response containing ImageNet class id and name. The updated app.py file will be now:

In [0]:
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    return jsonify({'class_id': 'IMAGE_NET_XXX', 'class_name': 'Cat'})

### **Step 4: Inference**
In the next sections we will focus on writing the inference code. This will involve two parts, one where we prepare the image so that it can be fed to DenseNet and next, we will write the code to get the actual prediction from the model.

Preparing the image
DenseNet model requires the image to be of 3 channel RGB image of size 224 x 224. We will also normalise the image tensor with the required mean and standard deviation values. You can read more about it here.

We will use transforms from torchvision library and build a transform pipeline, which transforms our images as required. You can read more about transforms here.

In [0]:
import io

import torchvision.transforms as transforms
from PIL import Image


def transforms_image(image_bytes):
  my_transforms = transforms.compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
  image = Image.open(io.BytesIO(image_bytes))
  return my_transforms(image.unsqueeze(0))



The above method takes image data in bytes, applies the series of transforms and returns a tensor. To test the above method, read an image file in bytes mode (first replacing ../_static/img/sample_file.jpeg with the actual path to the file on your computer) and see if you get a tensor back:

In [0]:
with open('path of Image/name.jpg' , 'rb') as f:
  image_bytes = f.read()
  tensor = transform_image(image_bytes = image_bytes)
  print(tensor)

### **Step 5: Prediction**
Now will use a pretrained DenseNet 121 model to predict the image class. We will use one from torchvision library, load the model and get an inference. While we’ll be using a pretrained model in this example, you can use this same approach for your own models. See more about loading your models in this tutorial.

In [9]:
from torchvision import models

# Make sure to pass `pretrained` as `True` to use the pretrained weights:
model = models.densenet121(pretrained = True)

# Since we are using our model only for inference, switch to `eval` mode:
model.eval()


def get_predictions(image_bytes):
  tensor = transform_image(image_bytes = image_bytes)
  outputs = model.forward(tensor)
  _, y_hat = outputs.max(1)
  return y_hat

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:01<00:00, 26.7MB/s]


The tensor y_hat will contain the index of the predicted class id. However, we need a human readable class name. For that we need a class id to name mapping. Download this file as imagenet_class_index.json and remember where you saved it (or, if you are following the exact steps in this tutorial, save it in tutorials/_static). This file contains the mapping of ImageNet class id to ImageNet class name. We will load this JSON file and get the class name of the predicted index.

In [0]:
import json

imagenet_class_index = json.load(open('/Path to JSON File/imagenet_class_index.json'))

def get_predictions(image_bytes):
  tensor = transform_image(image_bytes = image_bytes)
  outputs = model.forward(tensor)
  _ , y_hat = outputs.max(1)

  predicted_idx = str(y_hat)

  return imagenet_class_index[predicted_idx] 


Before using imagenet_class_index dictionary, first we will convert tensor value to a string value, since the keys in the imagenet_class_index dictionary are strings. 
We will test our above method:

In [0]:
with open('path of Image/name.jpg' , 'rb') as f:
  image_bytes = f.read()
  prediction = get_predictions(image_bytes)
  print(prediction)

You should get a response like this:

      `['n02124075', 'Egyptian_cat']`

The first item in array is ImageNet class id and second item is the human readable name.